In [33]:
import os
import re
import numpy as np
import matplotlib.image as mpimg
import pandas as pd
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ModelCheckpoint
import glob
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from PIL import Image
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report
import cv2 as cv
from keras.utils import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [73]:
input_data="chest_Xray/"
train_directory= input_data + "train/"
val_directory= input_data + "val/"
test_directory= input_data + "test/"

data_dir = train_directory
class_names=os.listdir(train_directory)
print(class_names)

['NORMAL', 'PNEUMONIA']


In [42]:
def get_accuracy_for_batch_size_augmented_data(train_folder,validation_folder, batch_size, image_size, optimizer, epochs, param_evaluated):
  print("Evaluating " + param_evaluated)
  # prepare iterators
  datagen = ImageDataGenerator(rescale=1.0/255.0)
  train_it = datagen.flow_from_directory(directory=train_folder,
    class_mode='binary', batch_size=batch_size, target_size=(image_size, image_size))
  val_it = datagen.flow_from_directory(directory=validation_folder,
    class_mode='binary', batch_size=batch_size, target_size=(image_size, image_size))
  test_it = datagen.flow_from_directory(test_directory,
    class_mode='binary', batch_size=batch_size, target_size=(image_size, image_size))
  
  # create model
  model = keras.Sequential()
  model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(image_size, image_size, 3)))
  model.add(keras.layers.MaxPooling2D((2, 2)))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(keras.layers.Dense(1, activation='sigmoid'))

  # compile model
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  # train the model
  model.fit(train_it,
                      validation_data = train_it,
                      steps_per_epoch = train_it.n//train_it.batch_size,
                      validation_steps = val_it.n//val_it.batch_size,
                      epochs=epochs, verbose=1)
  
  # evaluate model
  _, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
  return acc

In [79]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_model_accuracy(train_folder, validation_folder, batch_size, image_size, optimizer, epochs):
    # print("Evaluating " + param_evaluated)

    # Prepare iterators
    datagen = ImageDataGenerator(rescale=1.0 / 255.0)
    train_it = datagen.flow_from_directory(directory=train_folder,
                                           class_mode='binary', batch_size=batch_size,
                                           target_size=(image_size, image_size))
    val_it = datagen.flow_from_directory(directory=validation_folder,
                                         class_mode='binary', batch_size=batch_size,
                                         target_size=(image_size, image_size))
    test_it = datagen.flow_from_directory(test_directory,
                                          class_mode='binary', batch_size=batch_size,
                                          target_size=(image_size, image_size))

    # create model
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(image_size, image_size, 3)))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Define learning rate scheduler
    def lr_scheduler(epoch):
        lr = 0.001
        if epoch > 10:
            lr = 0.0001
        return lr

    # Create learning rate scheduler callback
    lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    # Train the model
    model.fit(train_it,
              validation_data=val_it,
              steps_per_epoch=train_it.n // train_it.batch_size,
              validation_steps=val_it.n // val_it.batch_size,
              epochs=epochs, verbose=1,
              callbacks=[lr_scheduler_callback])

    # Evaluate model
    _, acc = model.evaluate(test_it, steps=len(test_it), verbose=1)
    return acc

In [80]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class ModelAccuracyEstimator(BaseEstimator):
    def __init__(self, train_folder, validation_folder):
        self.train_folder = train_folder
        self.validation_folder = validation_folder

    def fit(self, X, y):
        batch_size, image_size, optimizer, epochs = X
        return get_model_accuracy(self.train_folder, self.validation_folder, batch_size, image_size, optimizer, epochs)

    def score(self, X, y):
        return self.fit(*X)  # Using *X to unpack the parameters

In [83]:
BATCH_SIZE_128 = 128
BATCH_SIZE_64 = 64
BATCH_SIZE_32 = 32
batch_size_array = [BATCH_SIZE_128, BATCH_SIZE_64, BATCH_SIZE_32]

OPTIMIZER_ADAM = 'adam'
OPTIMIZER_SGD = 'sgd'
OPTIMIZER_RMSPROP= 'rmsprop'
optimizer_array = [OPTIMIZER_ADAM, OPTIMIZER_SGD, OPTIMIZER_RMSPROP]

EPOCHS_5 = 5
EPOCHS_10 = 10
EPOCHS_20 = 20
epoch_array = [EPOCHS_5, EPOCHS_10, EPOCHS_20]

# Common image sizes include 64x64, 128x128, 28x28 (MNIST), and 224x224 (VGG-16).
IMAGE_SIZE_128 = 128
IMAGE_SIZE_64 = 64
IMAGE_SIZE_28 = 28
image_size_array = [IMAGE_SIZE_128, IMAGE_SIZE_64, IMAGE_SIZE_28]

In [85]:
# Define the parameter grid
param_grid = {
    'batch_size': batch_size_array,
    'image_size': image_size_array,
    'optimizer': optimizer_array,
    'epochs': epoch_array
}


# Perform grid search
best_acc = 0.0
best_params = {}
for batch_size in param_grid['batch_size']:
    print("Evaluating batch_size:", batch_size)
    for image_size in param_grid['image_size']:
        print("Evaluating image_size:", image_size)
        for optimizer in param_grid['optimizer']:
            print("Evaluating optimizer:", optimizer)
            for epochs in param_grid['epochs']:
                print("Evaluating epochs:", epochs)
                acc = get_model_accuracy(train_directory, val_directory, batch_size, image_size,
                                         optimizer, epochs)
                if acc > best_acc:
                    best_acc = acc
                    best_params = {'batch_size': batch_size, 'image_size': image_size,
                                   'optimizer': optimizer, 'epochs': epochs}

print("Best parameters:", best_params)
print("Best accuracy:", best_acc)

Evaluating batch_size: 128
Evaluating image_size: 128
Evaluating optimizer: adam
Evaluating epochs: 5
Found 5216 images belonging to 2 classes.
Found 17 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/5
40/40 [==============================] - 36s 876ms/step - loss: 5.7437 - accuracy: 0.7657 - lr: 0.0010
Epoch 2/5
40/40 [==============================] - 36s 902ms/step - loss: 0.1948 - accuracy: 0.9460 - lr: 0.0010
Epoch 3/5
40/40 [==============================] - 35s 871ms/step - loss: 0.0722 - accuracy: 0.9727 - lr: 0.0010
Epoch 4/5
40/40 [==============================] - 36s 891ms/step - loss: 0.0501 - accuracy: 0.9803 - lr: 0.0010
Epoch 5/5
5/5 [==============================] - 3s 598ms/step - loss: 0.8607 - accuracy: 0.7804
Evaluating epochs: 10
Found 5216 images belonging to 2 classes.
Found 17 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
40/40 [==============================] - 36s 883ms/step - loss: 5.4598